### Initial Setup

In [1]:
# imports
import numpy as np
import pandas as pd
import requests
import spotipy
import librosa
import json
import re 
import urllib.request

from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup

In [2]:
# spotify api creds
SPOTIFY_CLIENT_ID = ''
SPOTIFY_CLIENT_SECRET = ''

In [3]:
# function to get spotify token
def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    return auth_response_data['access_token']

In [4]:
# track details function
def get_track_details(spotify_id, token):
    sp = spotipy.Spotify(auth=token)
    track = sp.track(spotify_id)
    return track['name'], track['artists'][0]['name']

--- 

### Lyrics - Web Scraping Approach (Hit or Miss)

In [5]:
 
# function 1 to get the lyrics from spotify id
def get_lyrics_by_spotify_id(spotify_id): 

    spotify_token = get_spotify_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    song_title, artist = get_track_details(spotify_id, spotify_token)

    artist = artist.lower() 
    song_title = song_title.lower()

    # remove all except alphanumeric characters from artist and song_title 
    artist = re.sub('[^A-Za-z0-9]+', "", artist) 
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)

    # remove starting 'the' from artist e.g. the who -> who
    if artist.startswith("the"):     
        artist = artist[3:] 
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"

     
    try: 
        content = urllib.request.urlopen(url).read() 
        soup = BeautifulSoup(content, 'html.parser') 
        lyrics = str(soup) 
        # lyrics lies between up_partition and down_partition 
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->' 
        down_partition = '<!-- MxM banner -->' 
        lyrics = lyrics.split(up_partition)[1] 
        lyrics = lyrics.split(down_partition)[0] 
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').replace('<br/>','').strip()
        
        return lyrics     
    except Exception as e: 
        return "Exception occurred \n" +str(e)

### Lyrics - Free API (Adds jargon in the first line)

In [7]:
# function 2 to get the lyrics from spotify id
def get_lyrics_by_spotify_id(spotify_id): 

    spotify_token = get_spotify_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    song_title, artist = get_track_details(spotify_id, spotify_token)

    artist = str(artist) 
    song_title = str(song_title)

    url = 'https://api.lyrics.ovh/v1/' + artist + '/' + song_title

    # fetch lyrics
    response = requests.get(url)
    json_data = json.loads(response.content)
    lyrics = json_data['lyrics']

    return lyrics   

In [8]:
# example usage
spotify_id = 'https://open.spotify.com/track/1EdmLWe4HDbrw5CM0bOnxS?si=2888f7a69361427e'
lyrics = get_lyrics_by_spotify_id(spotify_id)

print(lyrics)

Paroles de la chanson Pretty People par Dillon Francis
I found what I'm searching for
Little bit, I'll take some more
Pretty people on the floor
Pretty people on the floor
I can taste all the lights
All the basses hittin' right
Don't know how we got so high
Don't know how we got so high

I found what I'm searching for
Little bit, I'll take some more
Pretty people on the floor
Pretty people on the floor
Pretty people on the floor
Pretty people on the floor
Pretty people, pretty people, pretty


It's going to my head
It's going to my head
It's going to my
It's going to my

And it feels like I'm alone with you
And it feels like I'm alone with you
And it feels like I'm alone with you
And it feels like I'm alone with you

I found what I'm searching for
Little bit, I'll take some more
Pretty people on the floor
Pretty people on the floor
I can taste all the lights
All the basses hittin' right
Don't know how we got so high

Don't know how we got so high

I found what I'm searching for
Little 

---